# Responsibility in terms of the Halpern-Pearl modified definition of actual causality

In a previous notebook TODO add link we introduced and implemented the Halpern-Pearl modified definition of actual causality. Here we implement the way Halpern used this notion to introduce his so-called *naive definition of responsibility*. We also briefly illustrate some reasons to think a somewhat more sophisticated notion is needed.

[Intuitions](##intuitions)
    
[Formalization](#formalization)

[Implementation](#implementation)

[Examples](#examples)

- [Comments on example selection](#comments-on-example-selection)
  
- [Voting](#voting)

- [Stone-throwing](#stone-throwing)

- [Firing squad](#firing-squad)


## Intuitions

The key idea here is that your responsibility for an outcome is to be measured in terms of how drastic a change would have to be made to the world for the outcome to depend counterfactually on your actions. However, the definition uses a fairly crude measure thereof, the minimal *number* of changes needed, where those numbers are individuated in terms of nodes. On one hand, if you are part of a cause, we count how many elements the cause has. On the other, we count the number of nodes that a witness set has. We add these two numbers for any combination of an actual cause and a witness set and we take the minimum, say $k$. Your responsibility is then $1/k$. 

## Formalization

The degree of responsibility of $X = x$ for $\varphi$ in $\langle M, \vec{u}\rangle$ is 0 if $X = x$ is not part of an actual cause of $\varphi$ in $\langle eM, \vec{u}\rangle$ according
to the modified HP definition. It is $1/k$ if there exists a cause $\vec{X} = \vec{x}$ of $\varphi$ and a witness $\vec{W}$ to $\vec{X}=\vec{x}$ being a cause of $\varphi$ in $\langle M, \vec{u}\rangle$ such that 
(a) $X=x$ is a conjunct in $\vec{X}= \vec{x}$, (b) $\vert \vec{W}\vert + \vert\vec{X}\vert$ = k$, and (c) $k$ is minimal such a number.


## Implementation

In [2]:
import functools

import numpy as np

import torch
from typing import Dict, List, Optional,  Union, Callable

import pandas as pd

import pyro
import pyro.distributions as dist

import random

from causal_pyro.indexed.ops import IndexSet, gather, indices_of, scatter
from causal_pyro.interventional.handlers import do
from causal_pyro.counterfactual.handlers import MultiWorldCounterfactual, Preemptions

TODO: ADD LINK
 We start with the implementation of actual causality described in the previous notebook.

In [3]:
class HalpernPearlModifiedApproximate:

    def __init__(
        self, 
        model: Callable,
        antecedents: Union[Dict[str, torch.Tensor], List[str]],
        outcome: str,
        witness_candidates: List[str],
        observations: Optional[Dict[str, torch.Tensor]],
        sample_size: int = 100,
        event_dim: int = 0
        ):
        
        self.model = model
        self.antecedents = antecedents
        self.outcome = outcome
        self.witness_candidates = witness_candidates
        self.nodes = antecedents + [outcome] + witness_candidates
        self.observations = observations
        self.sample_size = sample_size

        self.antecedents_dict = (
            self.antecedents if isinstance(self.antecedents, dict)
            else self.revert_antecedents(self.antecedents)
        )
    
        self.preemptions = {candidate: functools.partial(self.preempt_with_factual,
                                             antecedents = self.antecedents) for 
                                             candidate in self.witness_candidates}
        

    @staticmethod
    def revert_antecedents(antecedents: List[str]) -> Dict[str, Callable[[torch.Tensor], torch.Tensor]]:
        return {antecedent: (lambda v: 1 - v) for antecedent in antecedents}

    @staticmethod   
    def preempt_with_factual(value: torch.Tensor, *,
                          antecedents: List[str] = None, event_dim: int = 0):
    
        if antecedents is None:
            antecedents = []

        antecedents = [a for a in antecedents if a in indices_of(value, event_dim=event_dim)]

        factual_value = gather(value, IndexSet(**{antecedent: {0} for antecedent in antecedents}),
                                event_dim=event_dim)
            
        return scatter({
            IndexSet(**{antecedent: {0} for antecedent in antecedents}): factual_value,
            IndexSet(**{antecedent: {1} for antecedent in antecedents}): factual_value,
        }, event_dim=event_dim)
        
        
    def __call__(self, *args, **kwargs):
        with pyro.poutine.trace() as trace:
            with MultiWorldCounterfactual():
                with do(actions=self.antecedents_dict):
                    with Preemptions(actions = self.preemptions):
                        with pyro.condition(data={k: torch.as_tensor(v) for k, v in self.observations.items()}):
                            with pyro.plate("plate", self.sample_size):
                                self.consequent = self.model()[self.outcome]
                                self.intervened_consequent = gather(self.consequent, IndexSet(**{ant: {1} for ant in self.antecedents}))
                                self.observed_consequent = gather(self.consequent, IndexSet(**{ant: {0} for ant in self.antecedents}))
                                self.consequent_differs = self.intervened_consequent != self.observed_consequent   
                                pyro.factor("consequent_differs", torch.where(self.consequent_differs, torch.tensor(0.0), torch.tensor(-1e8)))
                            
        self.trace = trace.trace
        self.nodes_trace = {node: self.trace.nodes[node]['value'] for node in self.nodes}
        
        
         # slightly hacky solution for odd witness candidate sets
        if  isinstance(self.consequent_differs.squeeze().tolist(), bool):
            self.existential_but_for = self.consequent_differs.squeeze()
        else:
            #if (len(self.consequent_differs.squeeze().tolist() )>1):
            self.existential_but_for = any(self.consequent_differs.squeeze().tolist()                )  


        witness_dict = dict()
        if self.witness_candidates:
            witness_keys = ["__split_" + candidate for candidate in self.witness_candidates]
            witness_dict = {key: self.trace.nodes[key]['value']  for key in witness_keys}
            

        witness_dict['observed'] = self.observed_consequent.squeeze()
        witness_dict['intervened'] = self.intervened_consequent.squeeze()
        witness_dict['consequent_differs'] = self.consequent_differs.squeeze()

        # slightly hacky as above
        self.witness_df = pd.DataFrame(witness_dict) if self.witness_candidates else witness_dict

        if self.witness_candidates:
            self.witness_df['witness_size'] = self.witness_df[witness_keys].sum(axis = 1)
            satisfactory = self.witness_df[self.witness_df['consequent_differs'] == True]
            
        self.minimal_witness_size = satisfactory['witness_size'].min() if self.witness_candidates else 0
        self.responsibility_internal = 1/(len(self.antecedents) + self.minimal_witness_size)


This implementation is now used within another class definition, where, again, the main moves are in `def __call__`. Basically, we sample antecedents, leave other nodes (aside from the outcome) as witness candidates and pass the result to an actual causality evaluation, keeping track of minimal antecedent sets and the corresponding witness sizes. Then we find a minimum of the sum and use it in the denumerator.

In [4]:
class HalpernPearlResponsibilityApproximate:

    def __init__(
        self, 
        model: Callable,
        nodes: List,
        antecedent: str,
        outcome: str,
        observations: Dict[str, torch.Tensor], 
        runs_n: int 
    ):
        self.model = model
        self.nodes = nodes
        self.antecedent = antecedent
        self.outcome = outcome
        self.observations = observations
        self.runs_n = runs_n
        
        self.minimal_antecedents_cache = []
        self.antecedent_sizes = []
        self.existential_but_fors = []
        self.minimal_witness_sizes = []
        self.responsibilities = []
        self.HPMs = []

    def __call__(self):
        
        for step in range(1,self.runs_n):

            nodes = self.nodes
            if self.outcome in nodes:
                nodes.remove(self.outcome) 
            
            companion_size = random.randint(0,len(nodes))
            companion_candidates = random.sample(self.nodes, companion_size)
            witness_candidates = [node for node in self.nodes if 
                                node != self.antecedent and 
                                node != self.outcome and 
                                    node not in companion_candidates]

            HPM = HalpernPearlModifiedApproximate(
                model = self.model,
                antecedents = companion_candidates,
                outcome = self.outcome,
                witness_candidates = witness_candidates,
                observations = self.observations,
                sample_size = 1000)
            
            HPM()

            self.HPMs.append(HPM)


            if  HPM.existential_but_for:


                subset_in_cache = any([s.issubset(set(HPM.antecedents)) for s in self.minimal_antecedents_cache])
                if not subset_in_cache:
                    for s in self.minimal_antecedents_cache:
                        if set(HPM.antecedents).issubset(s):
                            self.minimal_antecedents_cache.remove(s)
                    self.minimal_antecedents_cache.append(set(HPM.antecedents))

                    if self.antecedent in HPM.antecedents:
                        self.antecedent_sizes.append(len(HPM.antecedents))
                        self.existential_but_fors.append(HPM.existential_but_for)
                        self.minimal_witness_sizes.append(HPM.minimal_witness_size)
                        self.responsibilities.append(HPM.responsibility_internal)


        self.denumerators = [x + y for x, y in zip(self.antecedent_sizes, self.minimal_witness_sizes)]

        self.responsibilityDF = pd.DataFrame(
            {"existential_but_for": [bool(value) for value in self.existential_but_fors],
                "antecedent_size": self.antecedent_sizes, 
                "minimal_witness_size": self.minimal_witness_sizes,
                "denumerator": self.denumerators,
                "responsibility": self.responsibilities
            }
            )
        if len(self.responsibilityDF['existential_but_for']) == 0:
            self.responsibility = 0
        else:
            min_denumerator = min(self.responsibilityDF['denumerator'])
            self.responsibility = 1/min_denumerator

 

## Examples

### Comments on example selection



- **Voting:** the example illustrates that parts of actual causes can share various degrees of responsibility for the outcome, without being actual causes.

- **Stone-throwing:** we illustrate responsibility calculations in one of the main running examples in the *Actual Causality* book by Halpern (2016).

- **Firing squad**   

### Voting

We discussed a similar model in a previous notebook TODO add link. This time we have eight voters involved in a binary majority voting procedure and we investigate the responsibility assigned to voter 0. The situation is analogous to the one discussed in the actual causality notebook: if your vote is decisive, you are an actual cause, and you're not an actual cause otherwise. What's your responsibility, though? 

In [6]:
def voting_model():
    u_vote0 = pyro.sample("u_vote0", dist.Bernoulli(0.6))
    u_vote1 = pyro.sample("u_vote1", dist.Bernoulli(0.6))
    u_vote2 = pyro.sample("u_vote2", dist.Bernoulli(0.6))
    u_vote3 = pyro.sample("u_vote3", dist.Bernoulli(0.6))
    u_vote4 = pyro.sample("u_vote4", dist.Bernoulli(0.6))
    u_vote5 = pyro.sample("u_vote5", dist.Bernoulli(0.6))
    u_vote6 = pyro.sample("u_vote6", dist.Bernoulli(0.6))
    u_vote7 = pyro.sample("u_vote7", dist.Bernoulli(0.6))

    vote0 = pyro.deterministic("vote0", u_vote0, event_dim=0)
    vote1 = pyro.deterministic("vote1", u_vote1, event_dim=0)
    vote2 = pyro.deterministic("vote2", u_vote2, event_dim=0)
    vote3 = pyro.deterministic("vote3", u_vote3, event_dim=0)
    vote4 = pyro.deterministic("vote4", u_vote4, event_dim=0)
    vote5 = pyro.deterministic("vote5", u_vote5, event_dim=0)
    vote6 = pyro.deterministic("vote6", u_vote6, event_dim=0)
    vote7 = pyro.deterministic("vote7", u_vote7, event_dim=0)


    outcome = pyro.deterministic("outcome", vote0 + vote1 + vote2 + vote3 + 
                                 vote4 + vote5 + vote6 + vote7 > 4)
    return {"outcome": outcome.float()}

In [11]:
# if you're one of five voters who voted for, you are an actual cause

# and your responsibility is 1 

voting5HPM = HalpernPearlModifiedApproximate(
    model = voting_model,
    antecedents = ["vote0"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(1,8)],
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
                        u_vote3=1., u_vote4=1., u_vote5=0,
                        u_vote6=0., u_vote7=0.),
    sample_size = 1000)

voting5HPM()

print(
voting5HPM.existential_but_for, 
voting5HPM.minimal_witness_size,
voting5HPM.responsibility_internal
)


True 0 1.0


In [15]:
everyone_voted_HPR = HalpernPearlResponsibilityApproximate(
    model = voting_model,
    nodes = [f"vote{i}" for i in range(0,8,)],
    antecedent = "vote0", outcome = "outcome",
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
    u_vote3=1., u_vote4=1., u_vote5=1., u_vote6 = 1., u_vote7 = 1.), 
    runs_n=500
    )

pyro.set_rng_seed(42)
everyone_voted_HPR()

In [16]:
# if everyone voted for, you are not an actual cause
# but the minimal antecedent size is 4, so your responsibility is 1/4

everyone_voted_HPR.responsibilityDF


,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility
0,True,4,0,4,0.250000
1,True,4,0,4,0.250000
2,True,5,0,5,0.200000
3,True,6,0,6,0.166667
4,True,4,0,4,0.250000
5,True,4,0,4,0.250000
6,True,6,0,6,0.166667
7,True,6,0,6,0.166667
8,True,4,0,4,0.250000
9,True,5,0,5,0.200000


In [17]:
# four people would need to change their votes
# to change the outcome
# so your responsibility is 1/4

everyone_voted_HPR.responsibility

0.25

In [18]:
# if only seven people voted for, 
# your responsibility changes to 1/3

seven_voted_for_HPR = HalpernPearlResponsibilityApproximate(
    model = voting_model,
    nodes = [f"vote{i}" for i in range(0,8,)],
    antecedent = "vote0", outcome = "outcome",
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
    u_vote3=1., u_vote4=1., u_vote5=1., u_vote6 = 1., u_vote7 = 0.), 
    runs_n=500
    )

pyro.set_rng_seed(42)
seven_voted_for_HPR()

In [19]:
seven_voted_for_HPR.responsibilityDF

,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility
0,True,4,0,4,0.250000
1,True,5,0,5,0.200000
2,True,3,0,3,0.333333
3,True,3,0,3,0.333333
4,True,3,0,3,0.333333
5,True,3,0,3,0.333333
6,True,3,0,3,0.333333
7,True,3,0,3,0.333333
8,True,3,0,3,0.333333
9,True,3,0,3,0.333333


In [20]:
# your responsibility is 1/3 as in this case
# it would be enough for three people to vote against
# to change the outcome

seven_voted_for_HPR.responsibility

0.3333333333333333

### Stone-throwing


We've already discussed the model in the actual causality notebook TODO add line Sally and Bill throw stones at a bottle, Sally throws first. Bill is perfectly accurate, so his stone would have shattered the bottle had not Sally's stone done it. The model is worth looking at, as the causal structure is less trivial. Again, we will see that responsibility judgment might to some extent disagree with actual causality.

In [22]:
def stones_model():        
    prob_sally_throws = pyro.sample("prob_sally_throws", dist.Beta(1, 1))
    prob_bill_throws = pyro.sample("prob_bill_throws", dist.Beta(1, 1))
    prob_sally_hits = pyro.sample("prob_sally_hits", dist.Beta(1, 1))
    prob_bill_hits = pyro.sample("prob_bill_hits", dist.Beta(1, 1))
    prob_bottle_shatters_if_sally = pyro.sample("prob_bottle_shatters_if_sally", dist.Beta(1, 1))
    prob_bottle_shatters_if_bill = pyro.sample("prob_bottle_shatters_if_bill", dist.Beta(1, 1))


    sally_throws = pyro.sample("sally_throws", dist.Bernoulli(prob_sally_throws))
    bill_throws = pyro.sample("bill_throws", dist.Bernoulli(prob_bill_throws))

    new_shp = torch.where(sally_throws == 1,prob_sally_hits , 0.0)

    sally_hits = pyro.sample("sally_hits",dist.Bernoulli(new_shp))

    new_bhp = torch.where(
            (
                bill_throws.bool()
                & (~sally_hits.bool())
            )
            == 1,
            prob_bill_hits,
            torch.tensor(0.0),
        )


    bill_hits = pyro.sample("bill_hits", dist.Bernoulli(new_bhp))

    new_bsp = torch.where(
            bill_hits.bool() == 1,
            prob_bottle_shatters_if_bill,
            torch.where(
                sally_hits.bool() == 1,
                prob_bottle_shatters_if_sally,
                torch.tensor(0.0),
            ),
        )

    bottle_shatters = pyro.sample(
            "bottle_shatters", dist.Bernoulli(new_bsp)
        )

    return {
            "sally_throws": sally_throws,
            "bill_throws": bill_throws,
            "sally_hits": sally_hits,
            "bill_hits": bill_hits,
            "bottle_shatters": bottle_shatters,
        }

stones_model.nodes = [
            "sally_throws",
            "bill_throws",
            "sally_hits",
            "bill_hits",
            "bottle_shatters",
        ]

In [24]:
pyro.set_rng_seed(101)
responsibility_stones_sally_HPR = HalpernPearlResponsibilityApproximate(
    model = stones_model,
    nodes = stones_model.nodes,
    antecedent = "sally_throws", outcome = "bottle_shatters",
    observations = {"prob_sally_throws": 1, 
                    "prob_bill_throws": 1,
                    "prob_sally_hits": 1,
                    "prob_bill_hits": 1,
                    "prob_bottle_shatters_if_sally": 1,
                    "prob_bottle_shatters_if_bill": 1,
                    "sally_throws": 1, "bill_throws": 1},
                      runs_n=100)

responsibility_stones_sally_HPR()

In [25]:
# note how minimal witness size becomes non-trivial here
# we only record different minimal difference-making scenarios
# there are two possible ones here

responsibility_stones_sally_HPR.responsibilityDF

,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility
0,True,2,0,2,0.5
1,True,1,1,2,0.5


In [26]:
# following Halpern
# Sally's responsibility is 1/2

responsibility_stones_sally_HPR.responsibility

0.5

In [27]:
# Halpern says in example 6.1.5. that 
# Billy has degree of responsibility 0
# for the bottle shattering, 
# since his throw was not a cause of the outcome.

# This argument doesn't work,
# as items that aren't actual causes can have responsibility 
# (see his own treatment of the voters case)

# In fact, a minimal difference-making scenario is one
# containing sally_throws and bill_throws and a null witness set

# another is 'bill_throws" and "sally_hits"

pyro.set_rng_seed(102)

responsibility_stones_bill_HPR = HalpernPearlResponsibilityApproximate(
    model = stones_model,
    nodes = stones_model.nodes,
    antecedent = "bill_throws", outcome = "bottle_shatters",
    observations = {"prob_sally_throws": 1, 
                    "prob_bill_throws": 1,
                    "prob_sally_hits": 1,
                    "prob_bill_hits": 1,
                    "prob_bottle_shatters_if_sally": 1,
                    "prob_bottle_shatters_if_bill": 1,
                    "sally_throws": 1, "bill_throws": 1},
                      runs_n=10)

In [28]:
pyro.set_rng_seed(101)
responsibility_stones_bill_HPR()

In [29]:
responsibility_stones_bill_HPR.responsibilityDF

,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility
0,True,2,0,2,0.5
1,True,2,0,2,0.5


In [25]:
step = 5

hpm = responsibility_stones_bill_HPR.HPMs[step]

print("antecedents:",  hpm.antecedents, 
      "existential_but_for:", hpm.existential_but_for)

step = 8

hpm = responsibility_stones_bill_HPR.HPMs[step]
print("antecedents:",  hpm.antecedents, 
      "existential_but_for:", hpm.existential_but_for)

print(
    responsibility_stones_bill_HPR.responsibility
)

antecedents: ['bill_throws', 'sally_throws'] existential_but_for: True
antecedents: ['bill_throws', 'sally_hits'] existential_but_for: True
0.5


### Firing squad

In [26]:

def firing_squad_model():
    probs = pyro.sample("probs", dist.Dirichlet(torch.ones(5)))

    who_has_bullet = pyro.sample("who_has_bullet", dist.OneHotCategorical(probs))

    mark0 = pyro.deterministic("mark0", torch.tensor([who[0] for who in who_has_bullet]), event_dim=0)
    mark1 = pyro.deterministic("mark1", torch.tensor([who[1] for who in who_has_bullet]), event_dim=0)
    mark2 = pyro.deterministic("mark2", torch.tensor([who[2] for who in who_has_bullet]), event_dim=0)
    mark3 = pyro.deterministic("mark3", torch.tensor([who[3] for who in who_has_bullet]), event_dim=0)
    mark4 = pyro.deterministic("mark4", torch.tensor([who[4] for who in who_has_bullet]), event_dim=0)

    dead = pyro.deterministic("dead", mark0 + mark1 + mark2 + mark3 + 
                                mark4  > 0)
    
    return {"probs": probs,
            "mark0": mark0,
            "mark1": mark1,
            "mark2": mark2,
            "mark3": mark3,
            "mark4": mark4, 
            "dead": dead}



In [28]:
pyro.set_rng_seed(102)

responsibility_loaded_HPR = HalpernPearlResponsibilityApproximate(
    model = firing_squad_model,
    nodes = ["mark" + str(i) for i in range(0,5)],
    antecedent = "mark0", outcome = "dead",
    observations = {"probs": torch.tensor([1., 0., 0., 0., 0.]),},
                      runs_n=50)


In [29]:

pyro.set_rng_seed(102)

responsibility_empty_HPR = HalpernPearlResponsibilityApproximate(
    model = firing_squad_model,
    nodes = ["mark" + str(i) for i in range(0,5)],
    antecedent = "mark1", outcome = "dead",
    observations = {"probs": torch.tensor([1., 0., 0., 0., 0.]),},
                      runs_n=50)

In [33]:
responsibility_loaded_HPR()
responsibility_loaded_HPR.responsibilityDF

,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility
0,True,1,0,1,1.0


In [35]:
# as we keep bullet's location constant
# nothing can make a difference to mark1's contribution

responsibility_empty_HPR()
responsibility_empty_HPR.responsibilityDF

,existential_but_for,antecedent_size,minimal_witness_size,denumerator,responsibility


In [36]:
responsibility_empty_HPR.responsibility

0